In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Загружаем данные

In [2]:
train_data = pd.read_parquet('train.parquet')
train_data

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


#### 1. Уберём данные у которых количество пропусков больше 25%

In [3]:
columns_with_omissions = []
for i in range(len(train_data.columns)):
    if train_data.isnull().sum()[i] > (len(train_data) * 0.25):
        columns_with_omissions.append(train_data.columns[i])
print(columns_with_omissions)    
print(len(columns_with_omissions))

['D_42', 'D_43', 'D_50', 'D_53', 'D_56', 'S_9', 'B_17', 'D_73', 'D_76', 'D_77', 'B_29', 'D_88', 'D_105', 'D_110', 'B_39', 'S_27', 'B_42', 'D_132', 'D_134', 'D_142']
20


In [4]:
for i in range(len(columns_with_omissions)):
    del train_data[columns_with_omissions[i]]
train_data.shape

(5531451, 170)

#### 2. Данные у которых количество повторяющихся элементов больше 75%

In [5]:
int_type = []
for i in range(2, len(train_data.columns)):
    type1 = str(train_data[train_data.columns[i]].dtypes)
    if type1 == 'int8' or type1 == 'int16':
        int_type.append(train_data.columns[i])
print(len(int_type))

95


In [6]:
columns_duplicate_elements = []
for i in range(len(int_type)):
    most_frequent_element = train_data[int_type[i]].value_counts()[train_data[int_type[i]].mode()[0]]
    if most_frequent_element > (len(train_data) * 0.75):
        columns_duplicate_elements.append(int_type[i])
columns_duplicate_elements.remove('R_10')
print(columns_duplicate_elements)
print(len(columns_duplicate_elements))

['R_2', 'D_49', 'S_6', 'R_4', 'R_5', 'D_65', 'D_66', 'B_22', 'D_72', 'D_78', 'D_79', 'R_8', 'R_9', 'R_11', 'D_81', 'R_13', 'D_83', 'R_15', 'D_84', 'R_16', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'B_31', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'D_106', 'R_26', 'D_108', 'D_109', 'D_111', 'D_116', 'D_120', 'D_123', 'D_125', 'D_126', 'D_127', 'B_41', 'R_28', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_143', 'D_145']
63


In [7]:
for i in range(len(columns_duplicate_elements)):
    del train_data[columns_duplicate_elements[i]]
train_data.shape

(5531451, 107)

#### 3. Данные с низким значением отклонения

In [8]:
low_std = []
for i in range(2, len(train_data.columns)):
    if 0.5 * train_data[train_data.columns[i]].mean() > train_data[train_data.columns[i]].std():
        low_std.append(train_data.columns[i])
low_std.remove('S_15')
print(low_std)    
print(len(low_std))

['P_2', 'D_46', 'P_3', 'D_54', 'D_59', 'S_11', 'D_63', 'D_68', 'R_12', 'S_25', 'R_27', 'D_112', 'D_121']
13


In [9]:
for i in range(len(low_std)):
    del train_data[low_std[i]]
train_data.shape

(5531451, 94)

#### 4. Данные с большим количеством выбросов

In [10]:
lot_emissions = []
for i in range(2, len(train_data.columns)):
    name = train_data[train_data.columns[i]]
    if name.quantile(q = 0.1) == name.quantile(q = 0.6) or 15 * name.quantile(q = 0.9) < name.max():
        lot_emissions.append(train_data.columns[i])
lot_emissions.remove('R_10')
print(lot_emissions)
print(len(lot_emissions))

['D_41', 'D_44', 'B_4', 'B_5', 'B_6', 'D_51', 'B_9', 'R_3', 'B_10', 'S_5', 'B_12', 'B_13', 'B_14', 'D_61', 'B_15', 'D_62', 'B_19', 'B_20', 'S_12', 'R_6', 'B_21', 'D_69', 'D_70', 'D_71', 'P_4', 'B_24', 'R_7', 'B_25', 'B_26', 'S_16', 'D_80', 'B_27', 'D_82', 'S_17', 'B_28', 'R_14', 'S_19', 'S_23', 'S_26', 'B_36', 'B_40', 'D_113', 'D_130', 'D_131', 'D_133', 'D_141', 'D_144']
47


In [11]:
for i in range(len(lot_emissions)):
    del train_data[lot_emissions[i]]
train_data.shape

(5531451, 47)

#### 5. Данные сильно коррелированные друг с другом

In [12]:
corr = train_data.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,D_39,B_1,B_2,R_1,S_3,B_3,D_45,D_47,D_48,B_7,B_8,D_52,B_11,S_7,S_8,D_55,D_58,D_60,D_64,B_16,B_18,S_13,S_15,B_23,D_74,D_75,R_10,B_33,S_22,S_24,D_102,D_103,D_104,D_107,B_37,B_38,D_114,D_115,D_117,D_118,D_119,D_122,D_124,D_128,D_129
D_39,1.000000,0.183361,-0.194840,0.194521,0.052359,0.173113,0.037497,-0.001865,0.172256,0.124983,-0.005390,-0.020366,0.177657,0.079295,0.151418,0.167894,0.131892,0.120307,0.035455,0.206226,-0.226726,0.152764,-0.080477,0.128160,0.115364,0.146386,0.108165,-0.245373,0.123435,0.136236,0.041957,0.066343,0.055930,0.069346,0.181794,0.187922,0.021166,-0.027180,0.010634,-0.022050,-0.021731,0.011572,0.055944,0.015443,0.037366
B_1,0.183361,1.000000,-0.631075,0.229492,0.155834,0.729890,-0.107227,-0.112942,0.420616,0.633427,0.130493,-0.118427,0.995574,0.141683,-0.016256,0.418881,0.369416,0.097250,0.068433,0.584735,-0.572362,-0.027131,0.045914,0.635437,0.303108,0.372148,0.128560,-0.565726,0.112936,0.122920,0.048813,0.084069,0.089294,0.081754,0.992915,0.579724,-0.059009,-0.079388,-0.047674,-0.075841,-0.074960,-0.055785,0.028920,-0.040057,-0.001810
B_2,-0.194840,-0.631075,1.000000,-0.303907,-0.235561,-0.713539,0.171038,0.185787,-0.571471,-0.591254,-0.205668,0.203787,-0.604082,-0.227735,0.184740,-0.541214,-0.520290,0.046295,-0.129893,-0.774889,0.850737,0.187930,-0.171293,-0.584502,-0.461296,-0.566242,-0.194562,0.912829,-0.098253,-0.104110,-0.061908,-0.136309,-0.135818,-0.129730,-0.634589,-0.670766,0.093817,0.117715,0.078549,0.109466,0.108113,0.084399,-0.046747,0.099944,0.028618
R_1,0.194521,0.229492,-0.303907,1.000000,0.212158,0.272562,-0.120006,-0.114655,0.388926,0.282846,0.146824,-0.115398,0.219360,0.210527,-0.077709,0.347770,0.321108,-0.067717,0.077694,0.247388,-0.312862,-0.052215,0.100239,0.285715,0.312531,0.333343,0.579127,-0.285919,0.059640,0.062877,0.031077,0.082057,0.080622,0.087431,0.229644,0.236547,-0.056191,-0.081881,-0.046740,-0.072048,-0.071186,-0.057758,0.050224,-0.064038,-0.024849
S_3,0.052359,0.155834,-0.235561,0.212158,1.000000,0.135866,-0.176044,-0.227772,0.279527,0.184443,0.210788,-0.170976,0.141809,0.904635,-0.398616,0.189601,0.196336,-0.091333,0.106962,0.139532,-0.239542,-0.187435,0.471225,0.170012,0.182993,0.201390,0.106048,-0.220156,0.016396,0.014730,-0.007437,0.045133,0.057568,0.043239,0.155398,0.158313,-0.170248,-0.098863,-0.138543,-0.090340,-0.089614,-0.144545,-0.062356,-0.166633,-0.112656
B_3,0.173113,0.729890,-0.713539,0.272562,0.135866,1.000000,-0.071076,-0.106266,0.518180,0.627605,0.095843,-0.141773,0.718244,0.104770,-0.102385,0.555345,0.488620,-0.009300,0.104628,0.724155,-0.653249,-0.128728,0.089813,0.634765,0.430173,0.501014,0.188350,-0.638151,0.093722,0.101419,0.060926,0.109474,0.103257,0.104151,0.733270,0.668338,-0.025075,-0.083567,-0.028874,-0.078795,-0.078025,-0.028269,0.061824,-0.047400,0.005941
D_45,0.037497,-0.107227,0.171038,-0.120006,-0.176044,-0.071076,1.000000,0.587870,-0.226455,-0.107913,-0.733183,0.245979,-0.088285,-0.201017,0.144931,-0.112457,-0.140197,0.060175,-0.109755,-0.072451,0.144365,0.123341,-0.151474,-0.089123,-0.141607,-0.145526,-0.089257,0.140314,-0.026027,-0.019136,-0.008828,-0.070563,-0.102437,-0.065796,-0.105605,-0.121143,0.266042,0.315484,0.144187,0.314360,0.310608,0.242987,0.081697,0.209887,0.138118
D_47,-0.001865,-0.112942,0.185787,-0.114655,-0.227772,-0.106266,0.587870,1.000000,-0.277972,-0.106176,-0.484473,0.237811,-0.095332,-0.259116,0.134805,-0.142724,-0.095631,-0.044136,-0.187349,-0.117540,0.155128,0.070851,-0.160234,-0.084224,-0.087141,-0.103602,-0.080448,0.173471,-0.040830,-0.039735,0.001142,-0.053192,-0.094455,-0.045138,-0.110972,-0.136545,0.356011,0.470661,0.142091,0.484437,0.482158,0.324172,0.152713,0.289747,0.173058
D_48,0.172256,0.420616,-0.571471,0.388926,0.279527,0.518180,-0.226455,-0.277972,1.000000,0.632696,0.265387,-0.331828,0.400080,0.294001,-0.146449,0.861719,0.640383,-0.158131,0.149903,0.531477,-0.683409,-0.093417,0.173997,0.629781,0.535901,0.575433,0.287629,-0.547480,0.098336,0.104562,0.065737,0.150295,0.164390,0.155288,0.420699,0.47944

In [14]:
corr = ['B_2', 'B_3', 'D_45', 'D_48', 'B_8', 'D_52', 'S_7', 'D_55', 'B_18', 'D_74', 'D_75', 'S_22', 'S_24', 
        'D_102', 'D_104', 'B_37', 'D_115', 'D_118', 'D_119', 'D_128', 'D_129']
for i in range(len(corr)):
    del train_data[corr[i]]
train_data.shape

(5531451, 26)

#### Записываем данные

In [15]:
table = train_data.set_index('customer_ID')

In [16]:
table.to_csv('train_data.csv')